In [85]:
!pip install python-scalpel


  Using cached python_scalpel-1.0b0-py3-none-any.whl.metadata (861 bytes)
Using cached python_scalpel-1.0b0-py3-none-any.whl (102 kB)


In [86]:
from scalpel.import_graph.import_graph import ImportGraph, Tree

In [87]:
import os
print("Current working directory:", os.getcwd())
target_dir = os.getcwd()  # Use the current working directory as the target directory
import_graph = ImportGraph(target_dir)
import_graph.build_dir_tree()



Current working directory: /Users/daniel.ellis/WIPwork/CMIP-LD/cmipld


In [88]:
all_leaf_nodes = import_graph.get_leaf_nodes()

module_imports = {}
for node in all_leaf_nodes:
    module_dict = import_graph.parse_import(node.ast)
    module_imports[import_graph.leaf2root(node)] = module_dict


module_imports

{'cmipld.version': {'typing': ['Tuple', 'Union']},
 'cmipld.__init__.py': {'locations': ['*'],
  'utils.server_tools.monkeypatch_requests': ['RequestRedirector'],
  'utils.server_tools.loader': ['Loader'],
  'utils': ['*']},
 'cmipld.gettest': {'pprint': ['pprint']},
 'cmipld.locations': {'pyld': ['jsonld'], 'tqdm': ['tqdm']},
 'cmipld.tests.__init__.py': {'jsonld': ['*']},
 'cmipld.tests.inputs': {},
 'cmipld.utils.urltools': {'urllib.parse': ['urlparse', 'urlunparse'],
  'os.path': ['relpath']},
 'cmipld.utils.checksum': {'copy': ['copy', 'deepcopy'],
  'collections': ['OrderedDict'],
  'git': ['getbranch',
   'getreponame',
   'getrepoowner',
   'getlastcommit',
   'getlasttag',
   'url',
   'url2io'],
  'locations': ['reverse_mapping']},
 'cmipld.utils.write06': {},
 'cmipld.utils.ldparse': {'collections': ['OrderedDict']},
 'cmipld.utils.io': {'git': ['toplevel']},
 'cmipld.utils.__init__.py': {'jsontools': ['*'],
  'io': ['*'],
  'write06': ['*'],
  'server_tools': ['*'],
  1: ['

In [ ]:
# import json
# from graphviz import Digraph

# # Replace this with loading from a JSON file or your original dict
# import_data = module_imports

# import os
# dot_path = "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/bin/"
# os.environ["PATH"] = dot_path + os.pathsep + os.environ["PATH"]

# # # Create a directed graph
# # dot = Digraph(comment="Module Import Graph", strict=True)
# # dot.attr(compound='true', rankdir='LR')

# dot = Digraph(comment="CMIPLD Module Import Graph", engine="sfdp")  # or engine="fdp", "neato"
# '''Engine	Best for
# dot	Hierarchical (default)
# neato	Force-directed layout
# fdp	Force-directed (for graphs)
# sfdp	Scalable force-directed
# twopi	Radial layout
# circo	Circular layout
# '''
# dot.attr(rankdir='TB')  # left-to-right layout
# dot.attr(compound='true', splines='true', overlap='false', concentrate='true')


# for module, imports in import_data.items():
#     dot.node(module)

#     for imp_module, items in imports.items():
#         if isinstance(imp_module, int):
#             # Skip numeric or malformed keys (e.g., 1 or 3 in the original data)
#             continue
#         try:
#             dot.edge(module, imp_module)
#         except Exception as e:
#             print(f"Error adding edge from {module} to {imp_module}: {e}")

# # Output to DOT file
# with open("import_graph.dot", "w") as f:
#     f.write(dot.source)

# # Optionally render as PNG
# dot.render("import_graph", format="pdf", cleanup=True)
# print("Graph written to import_graph.dot and rendered to import_graph.png")


Error adding edge from cmipld.utils.server_tools.__init__.py to None: 'NoneType' object has no attribute 'partition'
Error adding edge from cmipld.utils.git.__init__.py to None: 'NoneType' object has no attribute 'partition'
Graph written to import_graph.dot and rendered to import_graph.png


In [90]:
!open import_graph.pdf

In [91]:
# from graphviz import Digraph

# def visualize_import_graph(import_graph, output_file='import_graph'):
#     dot = Digraph(comment='Python Import Graph')

#     # Loop over all nodes and edges
#     for module, imports in import_graph.imports.items():
#         dot.node(module)  # Ensure the node exists
#         for imp in imports:
#             dot.edge(module, imp)

#     # Save and render
#     dot.render(output_file, format='png', cleanup=True)
#     print(f"Graph rendered to {output_file}.png")
    
# visualize_import_graph(module_imports, 'import_graph')

In [92]:
! /Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/bin/dot -Tpdf import_graph.dot -o graph.pdf

In [93]:
!open graph.pdf


In [100]:
import json
from graphviz import Digraph
import os

# Your import data
import_data = module_imports  # replace with your dict

# Setup dot path
dot_path = "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/bin/"
os.environ["PATH"] = dot_path + os.pathsep + os.environ["PATH"]

dot = Digraph(comment="CMIPLD Module Import Graph", engine="sfdp")
dot.attr(rankdir='TB')
dot.attr(compound='true', splines='true', overlap='false', concentrate='true')

def categorize_node(name):
    if name.endswith(".py"):
        return "pyfile"
    if name.startswith("cmipld."):
        level = name.count('.')
        if level == 1:
            return "top-level"
        else:
            return "nested"
    else:
        return "external"

all_nodes = set()
target_nodes = set()

for module, imports in import_data.items():
    all_nodes.add(module)
    for imp_module in imports.keys():
        if isinstance(imp_module, int):
            continue
        all_nodes.add(imp_module)
        target_nodes.add(imp_module)

styles = {
    "top-level": {"color": "blue", "shape": "box", "style": "filled", "fillcolor": "#c6dbef"},
    "nested": {"color": "darkgreen", "shape": "ellipse", "style": "filled", "fillcolor": "#a1d99b"},
    "external": {"color": "red", "shape": "diamond", "style": "filled", "fillcolor": "#fcbba1"},
    "isolated": {"color": "grey", "shape": "circle", "style": "filled", "fillcolor": "#eeeeee"},
    "pyfile": {"color": "orange", "shape": "rhombus", "style": "filled", "fillcolor": "#fdd0a2"},
}

for node in all_nodes:
    if not node:
        continue
    category = categorize_node(node)
    if node not in target_nodes and node not in import_data:
        category = "isolated"
    style = styles.get(category, {})
    dot.node(node,
             color=style.get("color"),
             shape=style.get("shape"),
             style=style.get("style"),
             fillcolor=style.get("fillcolor"))

for module, imports in import_data.items():
    for imp_module, items in imports.items():
        if isinstance(imp_module, int):
            continue
        try:
            dot.edge(module, imp_module)
        except Exception as e:
            print(f"Error adding edge from {module} to {imp_module}: {e}")

with open("import_graph.dot", "w") as f:
    f.write(dot.source)

dot.render("import_graph", format="pdf", cleanup=True)
print("Graph written to import_graph.dot and rendered to import_graph.pdf")


Error adding edge from cmipld.utils.server_tools.__init__.py to None: 'NoneType' object has no attribute 'partition'
Error adding edge from cmipld.utils.git.__init__.py to None: 'NoneType' object has no attribute 'partition'


Graph written to import_graph.dot and rendered to import_graph.pdf
